# 데이터 읽어오기

In [1]:
import os, re
import numpy as np
import tensorflow as tf
import glob

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

#여러 파일을 읽고 raw_corpus에 담기
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


# 데이터 정제

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue      #길이 0인문장은 건너뜀
    if sentence[-1] == ":": continue     #문장끝이 :인 문장은 건너뛸 것
        
    if idx>15: break  #길이 15초과일경우 반복문 종료
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof
You saw her bathing on the roof
Her beauty and the moonlight overthrew her
She tied you
To a kitchen chair
She broke your throne, and she cut your hair
And from your lips she drew the Hallelujah Hallelujah


# 전처리

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #소문자로 바꾸고 양쪽공백 지움
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #특수문자 양쪽에 공백
    sentence = re.sub(r'[" "]+', " ", sentence) #여러개의 공백은 하나의 공백으로
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #a-z 등등 이 아닌 문자를 공백으로 바꿈
    sentence = sentence.strip() # 다시 양쪽 공백을 제거
    sentence = '<start> ' + sentence + ' <end>' #문장 시작에는 <start>, 끝은 <end>추가
    return sentence

print(preprocess_sentence("@this! i¿s          samp!le"))

<start> this ! i ¿ s samp ! le <end>


In [4]:
corpus = [] #정제된 문장 모음

for sentence in raw_corpus:
    if len(sentence) == 0: continue    #원하지 않는 문장 생략
    if sentence[-1] == ":": continue  #대본의 경우 마지막에 ":" 이 붙음
        
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
corpus[:15] #정제된 것 15개 확인

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>',
 '<start> you saw her bathing on the roof <end>',
 '<start> her beauty and the moonlight overthrew her <end>',
 '<start> she tied you <end>',
 '<start> to a kitchen chair <end>',
 '<start> she broke your throne , and she cut your hair <end>']

# 토큰화과정

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=12000, 
    filters=' ',
    oov_token="<unk>"  # 12,000단어에 포함되지 못한 단어는 '<unk> =unknown으로 바꿈
    )                  #oov_token은 어휘목록에 포함되지 않은 단어를 대체할 때 사용
    
    tokenizer.fit_on_texts(corpus) #corpus를 이용해 tokenizer내부의 단어장 완성
    tensor = tokenizer.texts_to_sequences(corpus) #tokenizer를 이용해 corpus를 tensor로 변환
    #texts_to_sequences 메서드는 텍스트 안의 단어들을 숫자의 시퀀스로 변형시켜주는 것
    
    #입력 데이터의 시퀀스 길이를 맞춤, 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춤
    #문장 앞에 패딩을 붙여 길이를 맞추고싶다면 padding='pre'를 사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7faa7a4bccd0>


###### 이 부분에서 아무런 이상이 없는 줄 알았는데, maxlen 값을 하지 않아서 학습단계에서 아무것도 진행할 수 없었다. 영어라고 대충 읽지 말고 꼼꼼히 읽어서 이런 일이 일어나지 않도록 해야겠다.

# 단어 구축 확인

In [6]:
for idx in tokenizer.index_word:      #단어확인
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx>=15: break     #단어장 중 15번째의 단어까지 출력

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : t


In [7]:
src_input = tensor[:, :-1]  #마지막 부분을 잘라내서 문장 생성 = 패딩일 확률 높음
tgt_input = tensor[:, 1:]   #0번째 부터가 아닌 첫번째 부터 진행 = start를 자르기 위함

                            #패딩이 할당된 값은 0이 있기는 하지만,의미를 가지지 않음
print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


###### src_input = tensor[:, :-1] 부분은 end를 잘라내기 위함이라고 생각할 수있으나 pad값일 가능성이 높기 때문에 가장 마지막부분인 -1을 제외
###### tensor[:, :-1] 부분에서 콤마 앞부분의 콜론은 해당 문장을 선택한다는 뜻으로 보면 됨

###### 패딩이라는 단어가 꽤 생소해서 이것이 어떻게 작동하나 찾아봤는데, 데이터의 크기를 맞추기 위해 특정 값을 채우는 것을 패딩이라고 한다

###### src_input = tensor[:, :-1] 이 부분을 -1에서 자를 필요가 없이 범위를 지정하지 않으면 된다고 생각했으나, 팀원 분들께 여쭤보니 데이터 사이즈를 똑같이 맞추는데 중요한 역할을 한다고 하셨음

# 데이터 분리

In [8]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(
    src_input,
    tgt_input,
    test_size=0.2,
    random_state=42,
) 

# 데이터셋 객체 형성

In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE= 256                #BATCH_SIZE= 한번의 BATCH마다 주는 데이터 샘플의 SIZE
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어 + 0:<pad>를 포함하여 12,001개의 값을 가짐
VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset= dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

#### 데이터셋을 얻음으로써 데이터 다듬기 과정 끝

# 모델만들기

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size=256    #워드 벡터의 차원수, 단어가 추상적으로 표현되는 크기 [0.0, 1.0] 등을 의미
hidden_size = 1024   #모델에 얼마나 많은 일꾼을 둘 것인가?
model = TextGenerator(tokenizer.num_words+1, embedding_size, hidden_size)

In [11]:
#데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break
    
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 9.36345605e-05,  1.48246865e-04, -1.43573998e-04, ...,
         -2.89032410e-04, -3.02881963e-04,  5.74998267e-05],
        [ 1.31555398e-05,  2.61100591e-04, -2.20084083e-04, ...,
         -5.37787098e-04, -5.73951635e-04,  2.02090487e-05],
        [-1.40361444e-04,  3.51603754e-04, -2.41852744e-04, ...,
         -9.27612185e-04, -8.35872896e-04,  7.22916157e-05],
        ...,
        [ 5.21681679e-04, -1.63337623e-03,  2.77033367e-04, ...,
          1.93671859e-03,  5.73755067e-04, -3.03467666e-03],
        [ 5.42365015e-04, -1.78395933e-03,  2.93628458e-04, ...,
          2.06568115e-03,  6.72652735e-04, -3.25790583e-03],
        [ 5.59798733e-04, -1.91784464e-03,  2.99380743e-04, ...,
          2.15961062e-03,  7.44132267e-04, -3.44165321e-03]],

       [[ 9.36345605e-05,  1.48246865e-04, -1.43573998e-04, ...,
         -2.89032410e-04, -3.02881963e-04,  5.74998267e-05],
        [-2.91160868e-05,  3.99695913e-04, -2

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


# 학습중

In [13]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 122s 174ms/step - loss: 3.6126
Epoch 2/10
686/686 [==============================] - 119s 174ms/step - loss: 3.1393
Epoch 3/10
686/686 [==============================] - 120s 174ms/step - loss: 2.9478
Epoch 4/10
686/686 [==============================] - 120s 174ms/step - loss: 2.7997
Epoch 5/10
686/686 [==============================] - 119s 174ms/step - loss: 2.6711
Epoch 6/10
686/686 [==============================] - 119s 174ms/step - loss: 2.5539
Epoch 7/10
686/686 [==============================] - 119s 174ms/step - loss: 2.4452
Epoch 8/10
686/686 [==============================] - 119s 174ms/step - loss: 2.3428
Epoch 9/10
686/686 [==============================] - 120s 174ms/step - loss: 2.2465
Epoch 10/10
686/686 [==============================] - 120s 174ms/step - loss: 2.1553


### 이 부분에서 에러가나서 3시간 넘게 아무것도 하지 못했었다. 조원분들과 토론 결과 토큰 갯수를 15개 이하로 설정해야하는 것을 알게 되었다.

# 평가

In [23]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [24]:
#문장생성
generate_text(model,tokenizer, init_sentence="<start> he")

'<start> he s got a big ego hahaha <end> '